# Time-to-event analysis

This tab aims to investigate the impact of the treatment on the risk of an event occurring (often tumor progression or unfortunately death of the patient).

## 1. General

The most common graph is the representation of Kaplan-Meier curves. An important advantage of the Kaplan–Meier curve analysis is that the method can take into account some types of censored data. This representation is completed with a statistical test (ie., log-rank test) for significance investigation.

Note that before the log-rank test, a test is performed to investigate whether any variable in a Cox model breaks the proportional hazard assumption. In case this test reveals a violation of the assumption, the log-rank test result is not reported in legend and the p-value of the Schoenfeld residual test is provided instead.

In [1]:
import os
import sys
sys.path.append("C:\\Users\\Fabien Boux\\Code\\ClinLib")

from functions.config import Config
from clinlib.database import Database

with open('init.txt') as f:
    lines = f.readlines()
config = Config(os.path.join('ini', lines[0]))
config.read()

database = Database(config.get_value('database', section='PATH'), idlength=(int(config.get_value('id_length', section='OTHER')) if config.is_key('id_length') else 3))
database.add_resource({'metadata': os.path.join(config.get_value('database', section='PATH'), config.get_value('metadata', section='PATH'))})

import ipywidgets as widgets
from ipywidgets import interact, interactive
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
%matplotlib widget

from functions.graph import kaplan_meier_plot

if not config.is_key('followup'):
    config.extract_config_values('followup')
followup = int(int(config.get_value('followup')) / (365/12))

metric = (config.get_value('volume_label', section='OTHER') if config.is_key('volume_label') else 'Volume')
visits = (config.get_value('visits', section='OTHER') if config.is_key('visits') else None)
groups = (config.get_value('groups', section='OTHER') if config.is_key('groups') else None)


def plot_km(Event='OS', Followup=followup):
    if Event == 'PFS (adjusted)':
        fig = kaplan_meier_plot(database, event='PFS', adjust_ipfs=True, groups=groups,
                                visits=visits, followup_time=Followup, metric=metric)
    else:
        fig = kaplan_meier_plot(database, event=Event, adjust_ipfs=False, groups=groups,
                                visits=visits, followup_time=Followup, metric=metric)
    plt.show()

interact(plot_km, Event=['OS', 'PFS', 'PFS (adjusted)'], Followup=(0, round(followup*1.5), 1));

C:\ProgramData\Anaconda3\envs\NanoRad\lib\site-packages\statsmodels\compat\pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


interactive(children=(Dropdown(description='Event', options=('OS', 'PFS', 'PFS (adjusted)'), value='OS'), IntS…

## 2. Subgroup analysis

The next graph is a *forest plot*. This figure is used to easily account for benefit in specific subgroups. The benefit can be investigated using different metrics: hazard ratio (HR), risk ratio (RR), or time ratio (TR). For each line (each subgroup), the metric chosen is providen (*mark*) with a confidence interval (*line*) and the benefit is in favor of the first group (respectivelly, the second group) if the mark is positionned to the right (respectivelly, to the left) of the vertical "no effect" line. A significant result is obtained if there is no interception between the confidence interval and the "no effect" line. 

Note that this interpretation is valid for an event related to treatment benefit (eg., cured patient), for survival analysis invert right and left.
Note also that axis is reverted for time ratio to conserve the same reading direction than hazard ratio and risk ratio.

In [3]:
%matplotlib inline

from functions.graph import forest_plot

groups = (config.get_value('groups') if config.is_key('groups') else None)
list_selected = (config.get_value('monitored_metadata') if config.is_key('monitored_metadata') else [])
 
def plot_forest(Criteria='Hazard ratio (HR)'):
    if Criteria == 'Hazard ratio (HR)':
        model = 'HR'
    elif Criteria == 'Risk ratio (RR)':
        model = 'RR'
    elif Criteria == 'Time ratio (TR)':
        model = 'TR'
    else:
        model == 'HR'
    forest_plot(database, list_selected, model=model, groups=groups)
    
    if Criteria == 'Time ratio (TR)':
        plt.gca().invert_xaxis()
    
    plt.show()
    
interactive(plot_forest, Criteria=['Hazard ratio (HR)', 'Risk ratio (RR)', 'Time ratio (TR)'])

interactive(children=(Dropdown(description='Criteria', options=('Hazard ratio (HR)', 'Risk ratio (RR)', 'Time …